# Advent of Code 2020 - Day 3

In [39]:
enum class Terrain(val id: Char) {
    OPEN('.'),
    TREE('#'),
    ;

    companion object {
        private val idMap: Map<Char, Terrain> = Terrain.values().associateBy { it.id }
        fun fromChar(char: Char): Terrain = idMap[char]!!
    }
}

In [40]:
import java.io.File

typealias Topology = List<List<Terrain>>

val topology: Topology = File("Day3.input.txt")
    .bufferedReader()
    .readLines()
    .map { line ->line.map(Terrain::fromChar) }

## Part 1

Count all the trees you encounter on a slope of right 3, down 1 where your starting position is on top-left. For the following `Topology`, the number of encountered trees is 1. Note that, for reasons, the topology repeats forever to the right.
```
..##...
#...#..
.#....#
```

In [41]:
operator fun Pair<Int, Int>.plus(other: Pair<Int, Int>): Pair<Int, Int> = first + other.first to second + other.second

// First is Col/Down, Second is Row/Right
tailrec fun Topology.countTreesOnPath(position: Pair<Int, Int>, distance: Pair<Int, Int>, count: Int = 0): Int {
    val (col, row) = (position + distance).let { it.first to it.second % get(col).size }
    return if (col < size)
        when (get(col)[row]) {
            Terrain.TREE -> countTreesOnPath(col to row, distance, count + 1)
            else -> countTreesOnPath(col to row, distance, count)
        }
    else count
}

topology.countTreesOnPath(0 to 0, 1 to 3)

278

### Notes

The key part here is identifying that the topology repeats itself to the right which should immediately suggest the use of a modulus operator. Simply keep taking the modulus to find your X coordinate and step normally to get your Y coordinate.

Note that `tailrec` optimizes the recursive solution to an iterative solution so there isn't really a downside to using a recursive solution.

The time-complexity is `O(n)` where N is the size topology in the Y direction.

## Part 2

Do the same thing but with the following slopes and multiply their counts together:
- right 1, down 1
- right 3, down 1
- right 5, down 1
- right 7, down 1
- right 1, down 2

In [42]:
listOf(
    1 to 1,
    3 to 1,
    5 to 1,
    7 to 1,
    1 to 2,
)
    .map { (right, down) -> topology.countTreesOnPath(0 to 0, down to right) }
    .fold(1L) { acc, count -> acc * count }

9709761600

### Notes

Same problem, but you just have to do it a bunch of times. There was a small hiccup in that the final answer was larger than the size of an `Int` (`2_147_483_648`), so I needed to use a `Long`.